<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

В первый раз решил попробовать сделаьть сам проект через Колаб, т.к.  сожалению, Kernel постоянно вылетал при обработке данных. Нашел в интернете как установить библиотеки("pip install transformers" для возможной установки модель BERT и  "pip install catboost" для кэтбуста)

## Подготовка

In [ ]:
pip install transformers

     |████████████████████████████████| 3.4 MB 13.7 MB/s 
     |████████████████████████████████| 3.3 MB 36.9 MB/s 
     |████████████████████████████████| 596 kB 46.6 MB/s 
     |████████████████████████████████| 895 kB 42.2 MB/s 
     |████████████████████████████████| 61 kB 269 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
! pip install catboost

     |████████████████████████████████| 76.3 MB 44 kB/s 


Импортируем необходимые для исследования библиотеки. Для построения моделей для поиска лучшего значения метрики f1 мы возьмем CATBoost, LGBMClassifier и LogisticRegression

In [ ]:
import pandas as pd
import numpy as np
from tqdm import notebook
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer 

import torch
import transformers
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from pymystem3 import Mystem

from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from catboost import cv
import catboost as cb
from sklearn.model_selection import GridSearchCV
from catboost import Pool

from tqdm import notebook 
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Загрузим таблицу с комментариями, проверим количество строк и столбцов. При загрузке в колаб постоянно возникала ошибка, данные полностью не загружались, поэтому также прописал доп параметры.

In [ ]:
df_tweets = pd.read_csv('/content/sample_data/toxic_comments.csv',
                      error_bad_lines=False, engine="python")

Также, как уже сказал, в первый раз использую Colab, поэтому не совсем понимаю, сможет ли проверяющий загрузить исходный файл toxic_comments, кроме того прописал предоставления доступа к гугл - диску.  

In [ ]:
df_tweets.head(15)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [ ]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Нам будет необходимо перекодировать текст 

Рассмотрим процесс "очистки" на примере одного случайного предложения. Для этого воспользуемся модулем re, обозначим символы(буквы), которые мы оставим.

In [ ]:
df_tweets['text'] = df_tweets['text'].str.lower()

In [ ]:
df_tweets['text'][11]

'bbq \n\nbe a man and lets discuss it-maybe over the phone?'

In [ ]:
re.sub(r"[^a-zA-Z]",' ', df_tweets['text'][11])

'bbq   be a man and lets discuss it maybe over the phone '

Создадим корпус постов. Преобразуем столбец text в список текстов. Переведём тексты в стандартный для Python формат: кодировку Unicode. Выведем случайную строку

In [ ]:
corpus = df_tweets['text'].values.astype('U')
print(corpus[11])

bbq 

be a man and lets discuss it-maybe over the phone?


Очистим текст от лишних символов при помощи регулярных выражений. Для этого воспользуемся модулем **re**. Создадим функцию, которая очистит наш текст от лишних символов. В нее также пропишем функции join() и split() для устранения пробелов.
В первую очередь проеведем процесс лемматизации, для этого воспользуемся **"Wordnet"**  *(большая, свободно распространяемая и общедоступная лексическая база данных для английского языка с целью установления структурированных семантических отношений между словами)*

In [ ]:
wnl = WordNetLemmatizer()

In [ ]:
def text_clear(text):
    pattern = re.sub(r'[^a-zA-Z]', ' ', text)
    clear = pattern.split()
    lemm = []
    for i in range(len(clear)):
        lemm.append(wnl.lemmatize(clear[i]))
    return " ".join(lemm)

Выполним функции на "мешке" наших текстов. Так как количество строк большое, попробовал вставить функцию процессинга.

In [ ]:
%%time
for i in tqdm(range(len(corpus))):
    corpus[i] = text_clear(corpus[i])

100%|██████████| 159571/159571 [01:03<00:00, 2515.12it/s]

CPU times: user 1min 2s, sys: 505 ms, total: 1min 2s
Wall time: 1min 3s


Объединим нашу исходную таблицу и обновлненный текст, колонку со "старым" удалим.

In [ ]:
df_corpus = pd.DataFrame(corpus)
df_tweets['lemm_text'] = df_corpus[0]
df_tweets.drop('text', axis=1, inplace=True)
df_tweets

,toxic,lemm_text
0,0,explanation why the edits made under my userna...
1,0,d aww he match this background colour i m seem...
2,0,hey man i m really not trying to edit war it s...
3,0,more i can t make any real suggestion on impro...
4,0,you sir are my hero any chance you remember wh...
...,...,...
159566,0,and for the second time of asking when your vi...
159567,0,you should be ashamed of yourself that is a ho...
159568,0,spitzer umm there no actual article for prosti...
159569,0,and it look like it wa actually you who put on...


### Вывод по первой главе 

В данном разделе мы загрузили данные, посмотрели на таблицу. Она состоит из двух столбцов, пропусков в таблице нет. Рассмотрели на конкретном примере одного комментария, как работает очистка текста. После этого "очистили" текст при помощи создания функции. Новый столбец объединим с исходной таблицей, оставим столбец "Toxic" 

## 2. Обучение

In [ ]:
display(df_tweets.head(10))

,toxic,lemm_text
0,0,explanation why the edits made under my userna...
1,0,d aww he match this background colour i m seem...
2,0,hey man i m really not trying to edit war it s...
3,0,more i can t make any real suggestion on impro...
4,0,you sir are my hero any chance you remember wh...
5,0,congratulation from me a well use the tool wel...
6,1,cocksucker before you piss around on my work
7,0,your vandalism to the matt shirvington article...
8,0,sorry if the word nonsense wa offensive to you...
9,0,alignment on this subject and which are contra...


Вызовем функцию stopwords.words(), передадим ей аргумент 'english', то есть англоязычные стоп-слова:

In [ ]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words = stopwords)

Разобем данные на таргет и признаки. Целевым признаком в нашем случае является столбец "toxic".Разделим данные на обучющую и тестовую выборку, но до этого проведем векторизацию текста в features. Создадим счётчик, указав в нём стоп-слова(TfidfVectorizer).

Причина по которой провел векторизацию текста до разделения на обучающую и тестовую: после векторизации в тестовой и обучающей части признаков создавались матрицы различных масштабов, к сожалению, причину этого так и не смог определить и исправить

In [ ]:
features = df_tweets['lemm_text']
target = df_tweets['toxic']
random_state = 17
features_train_old, features_test_old, target_train, target_test = train_test_split(
  features, target, test_size=0.25, random_state=random_state, stratify = target)

In [ ]:
features_train = count_tf_idf.fit_transform(features_train_old)
features_test = count_tf_idf.transform(features_test_old)

Проверим размеры выборок 

In [ ]:
print('Размер обучающей выборки признаков, матрица :', features_train.shape)
print('Размер тестовой выборки признаков, матрица :', features_test.shape)
print('Размер обучающей выборки целевого признака', target_train.shape[0])
print('Размер тестовой выборки целевого признака',target_test.shape[0])

Размер обучающей выборки признаков, матрица : (119678, 133719)
Размер тестовой выборки признаков, матрица : (39893, 133719)
Размер обучающей выборки целевого признака 119678
Размер тестовой выборки целевого признака 39893


### 2.1 Логистическая регрессия

Первая модель, которую мы обучим, будет логистическая регрессия. Для начала воспользуемся GridSearch для поиска лучших параметров. Скорингом будет метрика f1. Также проверим, сколько времени потребуетсчя для поиска параметров

In [ ]:
%%time
regression = LogisticRegression(fit_intercept=True, 
                                random_state = random_state,
                                solver='liblinear',
                                n_jobs = -1
                               )
regression_parametrs = {'C': [0.1, 1, 10], 'class_weight': ['balanced', 'auto']}
regression_grid = GridSearchCV(
    regression, regression_parametrs, scoring='f1', cv=3)

CPU times: user 548 µs, sys: 0 ns, total: 548 µs
Wall time: 558 µs


In [ ]:
%%time
regression_grid.fit(features_train, target_train)
regression_model = regression_grid.best_estimator_
regression_model_predictions = regression_model.predict(features_test)
regression_params = regression_grid.best_params_
print('Лучшие параметры для логистической регрессии:', regression_params)

Лучшие параметры для логистической регрессии: {'C': 10, 'class_weight': 'balanced'}
CPU times: user 37.1 s, sys: 25.2 s, total: 1min 2s
Wall time: 33.4 s


In [ ]:
regression_f1 = round(f1_score(target_test, regression_model_predictions), 3) 
print(regression_f1)

0.762


Мы определили параметры для обучения, обучаем нашу модель и смотрим какое значение f1 мы получим

В целом Регрессия хорошо справилась с задачей показав результат лучше, чем нам определено в задаче, для нахождения параметров, обучения и предсказания потребовалось меньше минуты.

### 2.2 Light Gradient Boosted Machine

Следующую модель, которую мы используем будем LGBMClassifier.Проведем те же самые действия: поиск лучших параметров, обучение, предсказание и определения метрики f1. Здесь в качестве параметров будем искать глубину и "количество листьев"

In [ ]:
%%time
warnings.simplefilter(action = 'ignore', category = FutureWarning)
modelLight1 = LGBMClassifier(
    loss_function='f1', n_jobs = -1, random_state = random_state
    )
parameters_light = {'depth':[6,8], 'learning_rate':[0.2], 'num_leaves':[10,20]}
light = GridSearchCV(modelLight1, parameters_light,
            cv = 3,
            verbose = True)
light.fit(features_train, target_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
CPU times: user 8min 31s, sys: 1.62 s, total: 8min 32s
Wall time: 8min 33s


Выведем лучшие параметры для обучения

In [ ]:
modelLight = light.best_estimator_

In [ ]:
%%time
light_predict = modelLight.predict(features_test)
light_f1 = round(f1_score(target_test, light_predict),3)
print(light_f1)

0.76
CPU times: user 2.19 s, sys: 4.57 ms, total: 2.2 s
Wall time: 2.19 s


  Модель показала лучший результат чем предыдущая, однако времени на обучение и предсказание потребовалось гораздо больше (около 6 минут)

### 2.3 CatBoostClassifier

Наконец, возьмем Кэтбуст. Заранее скажу, что обучение и поиск параметров занимает очень большое количество времени, Кернел несколько раз ломался при его запуске. Установим самые минимальные параметры: 2 разреза кросс-валидации, количество итераций - 50. После этого найдем лучшие параметры, обучим модель и найдем f1

In [ ]:
%%time
final_cat = CatBoostClassifier(iterations = 50, random_state = random_state)
final_cat_parametrs = {'depth': [6, 8], 'learning_rate':[0.1]}
final_cat_grid = GridSearchCV(final_cat, final_cat_parametrs,
                              scoring='f1', cv = 2)
final_cat_grid.fit(features_train, target_train, verbose = 25)   

0:	learn: 0.5967894	total: 2.51s	remaining: 2m 3s
25:	learn: 0.2170912	total: 37.3s	remaining: 34.4s
49:	learn: 0.1932761	total: 1m 10s	remaining: 0us
0:	learn: 0.5977495	total: 2.46s	remaining: 2m
25:	learn: 0.2148175	total: 36.6s	remaining: 33.8s
49:	learn: 0.1901197	total: 1m 9s	remaining: 0us
0:	learn: 0.5963648	total: 6.28s	remaining: 5m 7s
25:	learn: 0.2089863	total: 1m 44s	remaining: 1m 36s
49:	learn: 0.1842745	total: 3m 18s	remaining: 0us
0:	learn: 0.5933442	total: 5.26s	remaining: 4m 17s
25:	learn: 0.2052600	total: 1m 44s	remaining: 1m 36s
49:	learn: 0.1808123	total: 3m 17s	remaining: 0us
0:	learn: 0.5926810	total: 9.61s	remaining: 7m 50s
25:	learn: 0.2060613	total: 2m 50s	remaining: 2m 37s
49:	learn: 0.1824393	total: 5m 23s	remaining: 0us
CPU times: user 29min 42s, sys: 23.2 s, total: 30min 5s
Wall time: 15min 55s


In [ ]:
cb_params = final_cat_grid.best_params_
cb_params
cat = final_cat_grid.best_estimator_

In [ ]:
%%time
cat_predictions = cat.predict(features_test)
cat_f1 = round(f1_score(target_test, cat_predictions),3)
cat_f1

CPU times: user 658 ms, sys: 26.9 ms, total: 685 ms
Wall time: 456 ms


## 3. Выводы

In [ ]:
finall = {'LogisticRegression':np.array([regression_f1, '5.13s']),
           'LGBMRegressor':np.array([light_f1, '8min 57s']),
           'Catboost':np.array([cat_f1, '15min 17s'])}
finall = pd.DataFrame(finall, index = ['Значение F1','Время обучения и предсказание'])
finall

,LogisticRegression,LGBMRegressor,Catboostr
Значение F1,0.762,0.76,0.61
Время обучения и предсказание,5.13s,3min 57s,4min 17s


В первой главе произведена загрузка данных, установлены необходимые библиотеки. Для анализа данных предоставлены данные, содержащие комментарии пользователей к товарам, доступным для приобретения в интернет-магазине «Викишоп». Далее в той же главе "очистили" комментарии и лемматизировали текст.

После этого разбили данные на тестовую и обучающую выборки, обозначили целевой признак (в нашей задаче его роль взял стоблец "Toxic"). Использовали TfidfVectorizer, указав стоп слова и изпользовав английский словарь.

Далее мы обучили 3 модели: Логистическую регрессию, LGBMClassifier и CatBoostClassifier. Для начала мы при помощи GridSearch подбирали для каждой модели гиперпараметры, после этого обучили модель и определили метрику f1 для каждой модели. В конце составлена небольшая таблица итогов по каждой модели. Исходя из этой таблицы, лучшей моделью стала Логистическая регрессия. Потребовалось менее минуты для поиска гиперпараметров и обучения модели, результат f1 немного уступает следующей модели - LGBMClassifier(однако с задачей преодоления порога в   0.75 он справился). 
LGBMClassifier показал результат выше, однако времени было потрачено гораздо болше. CatBoostClassifier показал самый худший результат, как по времени подбора гиперпараметров и обучения, так и по результату метрики f1.
 К сожалению, BERT установить не удалось, столкнулся с проблемой установок самих библиотек. 

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны